In [1]:
import boto3
from boto3 import client
import json
import pandas as pd

In [37]:
s3 = boto3.resource('s3')

In [55]:
j_file = "train_test_polygons.json"

with open(j_file, 'r') as j:
    train_test = json.loads(j.read())


# Export Chip List

In [54]:
import json

with open('yes_chips_s3.json', 'w') as fp:
    json.dump(chips, fp)


In [58]:
chips[0:3]

['chips/cloudfree-merge-polygons/yes/Habitation/100/101/101_1000_3000.tif',
 'chips/cloudfree-merge-polygons/yes/Habitation/100/101/101_1000_3100.tif',
 'chips/cloudfree-merge-polygons/yes/Habitation/100/101/101_1000_400.tif']

# Train / Test Copy Based on JSON Train / Test Chip List

In [59]:
def train_test_s3_copy(chips, j_file, bucket_name='canopy-production-ml',
                       base_path='chips/cloudfree-merge-polygons/split/'):
    
    bucket = s3.Bucket(bucket_name)
    
    with open(j_file, 'r') as j:
        
        train_test_file = json.loads(j.read())
        
#     for split in list(train_test.keys()):
        
#         for polygon in train_test[split]:

    length = len(chips)
            
    for i, chip in enumerate(chips, 1):
        print(f'Processing chip {i} of {length}', end='\r', flush=True)

        CopySource = {
            'Bucket': bucket_name,
            'Key': chip
        }

        polygon_id = int(chip.split("/")[5])

        filename = chip.split('/')[-1]

        if polygon_id in train_test_file["test"]:
            train_test = 'test'

            #s3.Object(bucket, base_path + 'test/').copy_from(CopySource=bucket + chip.key)

        else:
            train_test = 'train'

            #s3.Object(bucket, base_path + 'train_val/').copy_from(CopySource=bucket + chip.key)

        new_key = f'{base_path}{train_test}/{polygon_id}/{filename}'

        bucket.copy(CopySource, new_key)

In [61]:
j_file = "train_test_polygons.json"
train_test_s3_copy(chips,j_file)

# Train / Val Split

## S3 Master Actions

In [119]:
def s3_actions(bucket, in_path=None, out_path=None, copy_list=None, copy_dir=False, delete=False, stop_at=None, chip_list=False, chip_count=False):
    s3 = boto3.resource('s3')
    my_bucket = s3.Bucket(bucket)
    objs = []
    total_files = 0 
    

    
    if copy_dir or delete or chip_list or chip_count:
        for obj in my_bucket.objects.filter(Prefix=in_path):
            total_files += 1 
                objs.append(obj)
            if total_files == stop_at:
                break
        
    tot_objs = len(objs)
        
    if delete:
        for index,obj in enumerate(objs,1):
            print(f"deleting {index} of {tot_objs}", end='\r', flush=True)
            obj.delete()
            
    if chip_list:
        return objs
    
    if chip_count:
        return total_files
    
    if copy_list:
        total_copy_list = len(copy_list)

        for index,obj in enumerate(copy_list,1):
            
            print(f"copying {index} of {total_copy_list}", end='\r', flush=True)
            old_key = in_path + obj
            new_key = out_path + obj
            CopySource = {
                'Bucket': bucket,
                'Key':old_key}
            my_bucket.copy(CopySource, new_key)
            
            
            

In [54]:
bucket = 'canopy-production-ml'
paths = ['chips/cloudfree-merge-polygons/split/train_copy/',
         'chips/cloudfree-merge-polygons/split/train/',
        'chips/cloudfree-merge-polygons/split/test/',
        'chips/cloudfree-merge-polygons/yes/',
        'chips/cloudfree-merge-polygons/split/val/']
    
dirs = [s3_actions(bucket,i,chip_list=True) for i in paths[:2]]

# s3_actions(bucket,paths,chip_count=True)

In [44]:
len(train_chips)

34071

In [82]:
dirs[0][1].key.split("/")[-2:]

['1', '1_1200_1800.tif']

In [77]:
filenames_t_copy = []
filenames_t = []

In [85]:
f = [filenames_t.append("/".join(i.key.split("/")[-2:])) for i in dirs[1]]
f = [filenames_t_copy.append("/".join(i.key.split("/")[-2:])) for i in dirs[0]]

In [92]:
new_chips = list(set(filenames_t_copy) ^ set(filenames_t))

In [98]:
# new_chips.remove("/")
new_chips.remove("train/")

In [107]:
new_chips

['1/1_1400_1100.tif',
 '1/1_1200_900.tif',
 '10/10_1700_3400.tif',
 '10/10_1700_2300.tif',
 '1/1_1300_2000.tif',
 '10/10_1700_3600.tif',
 '1/1_1500_1100.tif',
 '1/1_1400_900.tif',
 '10/10_1700_2000.tif',
 '1/1_1300_1600.tif',
 '1/1_1400_1700.tif',
 '10/10_1800_2400.tif',
 '10/10_1700_2900.tif',
 '1/1_1300_1400.tif',
 '1/1_1500_1400.tif',
 '1/1_1400_1000.tif',
 '10/10_1800_2500.tif',
 '1/1_1500_1500.tif',
 '1/1_1200_1900.tif',
 '10/10_1700_2500.tif',
 '1/1_1500_1200.tif',
 '1/1_1400_1600.tif',
 '1/1_1300_800.tif',
 '10/10_1700_2600.tif',
 '1/1_1300_1100.tif',
 '1/1_1400_700.tif',
 '10/10_1700_3200.tif',
 '1/1_1400_1300.tif',
 '1/1_1400_1500.tif',
 '1/1_1400_600.tif',
 '10/10_1800_2600.tif',
 '1/1_1400_1800.tif',
 '1/1_1300_700.tif',
 '1/1_1300_900.tif',
 '1/1_1200_2100.tif',
 '10/10_1600_3900.tif',
 '10/10_1700_3500.tif',
 '1/1_1400_2000.tif',
 '10/10_1800_2000.tif',
 '1/1_1200_1800.tif',
 '10/10_1700_2800.tif',
 '1/1_1300_1800.tif',
 '10/10_1700_2100.tif',
 '1/1_1200_2000.tif',
 '1/1_1

In [120]:
s3_actions(bucket,in_path="chips/cloudfree-merge-polygons/split/train_copy/", 
           out_path="chips/cloudfree-merge-polygons/split/train/",copy_list=new_chips)

# Train / Val Split - Copy Train Chips to Val / Delete Same Chips from Train

In [45]:
def train_val_split(train_chips,bucket_name='canopy-production-ml',
                    in_base_path='chips/cloudfree-merge-polygons/split/',
                    out_base_path='chips/cloudfree-merge-polygons/split/val/',
                   copy=True,
                   delete=True):
    
    bucket = s3.Bucket(bucket_name)
    train_chips = train_chips.copy()
    dir_dict = {}
    
    for chip_1 in train_chips:
        
        chip_key = chip_1.key
        poly_id = chip_key.split("/")[-1].split("_")[0]
        
        if poly_id in dir_dict.keys():
                
            dir_dict[poly_id].append(chip_key)
                
        else:

            dir_dict[poly_id] = [chip_key]
            
    
    key_num = len(dir_dict.keys())
    
    for i_1,key in enumerate(dir_dict.keys(),1):
        
        dir_len = len(dir_dict[key])
        val_slice = dir_len // 5 
        val_chips = dir_dict[key][:val_slice]
        val_num = len(val_chips)
        
        for i_2,val_chip_key in enumerate(val_chips,1):
            
            print(f"Copying chip {i_2} of {val_num} in polygon {i_1} of {key_num}", end='\r', flush=True)
            polygon_id = val_chip_key.split("/")[-1].split("_")[0]
            filename = val_chip_key.split('/')[-1]
            new_key = f'{out_base_path}{polygon_id}/{filename}'
            
            if copy:
                
                CopySource = {
                'Bucket': bucket_name,
                'Key':val_chip_key}
                
                bucket.copy(CopySource, new_key)
            
            if delete:
            
                obj = s3.Object(bucket_name, val_chip_key)
                obj.delete()
    

In [46]:
train_val_split(train_chips)

In [81]:
val_chips = s3_list(dir_name='val',bucket='canopy-production-ml')

In [14]:
train_chips[0:3]

[s3.ObjectSummary(bucket_name='canopy-production-ml', key='chips/cloudfree-merge-polygons/split/train/'),
 s3.ObjectSummary(bucket_name='canopy-production-ml', key='chips/cloudfree-merge-polygons/split/train/1/1_1000_1000.tif'),
 s3.ObjectSummary(bucket_name='canopy-production-ml', key='chips/cloudfree-merge-polygons/split/train/1/1_1000_1100.tif')]

# Copy Directory

In [12]:
def copy_dir(file_list,
             out_base_path='chips/cloudfree-merge-polygons/split/train_copy/',
             bucket_name='canopy-production-ml'):

    bucket = s3.Bucket(bucket_name)

    dir_num = len(file_list)

    for i_2,chip in enumerate(file_list,1):

        chip_key = chip.key

        print(f"Copying chip {i_2} of {dir_num}", end='\r', flush=True)

        CopySource = {
            'Bucket': 'canopy-production-ml',
            'Key':chip_key}

        polygon_id = chip_key.split("/")[-1].split("_")[0]

        filename = chip_key.split('/')[-1]

        new_key = f'{out_base_path}{polygon_id}/{filename}'

        bucket.copy(CopySource, new_key)

In [13]:
copy_dir(train_chips[1:])

In [34]:
for chip in train[1:2]:
    
    obj = s3.Object('canopy-production-ml', chip.key)
    
#     obj.delete()

NameError: name 'val_chips' is not defined

In [12]:
def s3_actions(bucket, path, copy=False, delete=False, stop_at=None, chip_list=False, chip_count=False):
    s3 = boto3.resource('s3')
    my_bucket = s3.Bucket(bucket)
    objs = []
    total_files = 0 

    for obj in my_bucket.objects.filter(Prefix=path):
        
        total_files += 1 
        
        if copy or delete:
        
            objs.append(obj)

        if total_files == stop_at:

            break
        
    tot_objs = len(objs)
        
    if delete:

        for index,obj in enumerate(objs,1):
            
            print(f"deleting {index} of {tot_objs}", end='\r', flush=True)
            
            obj.delete()
            
    if chip_list:
        
        return objs
    
    if chip_count:
        
        return total_files

In [21]:
bucket = 'canopy-production-ml'
paths = ['chips/cloudfree-merge-polygons/split/train_copy/',
        'chips/cloudfree-merge-polygons/split/test/',
        'chips/cloudfree-merge-polygons/yes/',
        'chips/cloudfree-merge-polygons/split/val/']
    
# dirs = [s3_actions(bucket,i,chip_count=True) for i in paths]

s3_actions(bucket,paths[3],chip_count=True)

6795

In [14]:
dirs

[34071, 7707, 49133]

In [15]:
dirs[0] + dirs[1]

41778